In [112]:
import gc
import re
import os
import copy
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from skimage.io import imread
from sklearn.neighbors import KNeighborsClassifier as ks

In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
import cv2
arr=[]
labels = []
num_nonfaces =[]
image1 =[]
i=0
nonfaces=0
for root,di,files in os.walk("/content/drive/MyDrive/Colab Notebooks/dataset2"): 
  for name in files:
    if name.endswith("jpg") or name.endswith("pgm") :
      root_folder = os.path.split(root) ##   
      if name.endswith("jpg"): 
        nonfaces = nonfaces + 1 
        #image1 = io.imread(os.path.join(root,name))
        image1 = cv2.imread(os.path.join(root,name), 0)
      elif name.endswith("pgm") :
        image1 = imread(os.path.join(root,name))  
      res=re.findall('(\d+)',root_folder[1]) ##
      #image1 = image.imread(os.path.join(root,name))
      image1.resize((92, 112))
      image1=image1.flatten(order='C')
      #print(image1)
      arr.append(image1)
      labels.append(int(res[0]))
      i=i+1
num_nonfaces.append( nonfaces)     
arr=np.array(arr)   
labels=np.array(labels)    
print(labels.shape)       
print(arr.shape)
#print(labels)      

(600,)
(600, 10304)


In [126]:
print(labels.shape)       
print(arr.shape)
#print(labels)

(844,)
(844, 10304)


In [ ]:
# adding the data to a dataframe
df = pd.DataFrame(arr) 
df['label'] = pd.DataFrame(labels)
# sorting the data according to labels
df.sort_values("label", inplace=True)
df.tail(10)

In [116]:
# seperating images from labels
Y = np.array(df['label'])
df = df.drop(axis=1, labels="label")

In [117]:
X_train = []
X_test =  []  
labels_train = []
labels_test =  []  

for i in range(0,600):
    if((i%2==0)):
       X_test.append(arr[i,:])
       labels_test.append(labels[i])
    else:
       X_train.append(arr[i,:])
       labels_train.append(labels[i])
 
print(X_test[0])
print(labels_test[0])

[157 156 153 ... 141 141 133]
1


In [118]:
def LDA(arr, Y=None, noOfDims=None):
    # initializing the variables
    summ, lenn = 0, 0
    # computing the mean for each class
    mu = []
    Z =  []
    if nonfaces >200:
      arr2=np.concatenate((arr[0:199,:],arr[600:,:]))
      mu.append(np.mean(arr2, axis=0, keepdims=True).T) 
    else: 
      mu.append(np.mean(arr[0:199,:], axis=0, keepdims=True).T) 
    mu.append(np.mean(arr[200:599,:], axis=0 , keepdims=True).T)    
    # computing the between class scatter matrices
    SB = np.matmul((mu[0]-mu[1]), (mu[0]-mu[1]).T)
    # centering the class matrices
    if nonfaces >200:
        Z.append( (arr2 - mu[0].T)) 
    else:
        Z.append((arr[0:200,:] - mu[1].T))    
    Z.append((arr[200:599,:] - mu[1].T))
    S = 0
    for i in range(len(Z)): S += np.matmul(Z[i].T, Z[i])
    # performing eigenvalue decomposition
    eigValues, eigVectors = np.linalg.eigh(np.matmul(np.linalg.inv(S), SB))
    # returning the dominant eigen vectors
    return eigVectors[::-1][:noOfDims].T

    

In [119]:
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES
W = LDA(arr, noOfDims=2)
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES

In [ ]:
# saving the LDA output for later use
file_name = 'LDA_output.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(W, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "LDA_output.pkl"


In [86]:
print("After LDA")
X_train_red, X_test_red = projectData(np.array(X_train), np.array(X_test), W)
#reg = ks().fit(X_train_red, labels_train)
#train_score = reg.score(X_train_red, labels_train)
#test_score = reg.score(X_test_red, labels_test)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, labels_train)
pred = knn.predict(X_test)
print('200',accuracy_score(pred,labels_test))
#acc.append(accuracy_score(pred,labels_test))
#print(X_test_red)
print("Training Set Accuracy: {}\nTesting  Set Accuracy: {}".format(train_score, test_score))

After LDA
200 0.9933333333333333
Training Set Accuracy: 0.76
Testing  Set Accuracy: 0.7733333333333333


In [ ]:
file_name = 'LDA_output.pkl'
with open(file_name, 'rb') as file:
    W  = pickle.load(file)
    print(f'Object successfully loaded from "{file_name}"')

[[-0.00480145  0.00697254]
 [-0.00466157  0.00676886]
 [-0.00391163  0.00567956]
 ...
 [-0.00378211  0.00549338]
 [-0.00464662  0.00674866]
 [-0.00475303  0.00690263]]
Object successfully loaded from "LDA_output.pkl"


In [120]:
def projectData(X_train, X_test, W):
    X_train = np.array([np.matmul(W.T, X_train[i]) for i in range(X_train.shape[0])])
    X_test = np.array([np.matmul(W.T, X_test[i]) for i in range(X_test.shape[0])])
    return X_train, X_test

In [123]:
labels2 = []
arr2 =[]

for root,di,files in os.walk("/content/drive/MyDrive/Colab Notebooks/dataset3/11"): 
  for name in files:
    if name.endswith("jpg") or name.endswith("jpeg") or name.endswith("pgm") :
      root_folder = os.path.split(root) ##   
      if name.endswith("jpg") or name.endswith("jpeg"): 
        nonfaces = nonfaces + 1
        labels2.append(1)
        #image1 = io.imread(os.path.join(root,name))
        image1 = cv2.imread(os.path.join(root,name), 0)
      elif name.endswith("pgm") :
        image1 = image.imread(os.path.join(root,name))
        labels2.append(2)  
      #res=re.findall('(\d+)',root_folder[1]) ##
      #image1 = image.imread(os.path.join(root,name))
      image1.resize((92, 112))
      image1=image1.flatten(order='C')
      #print(image1)
      arr2.append(image1)
     # labels.append(int(res[0]))
      i=i+1
      
num_nonfaces.append( nonfaces)           
arr2=np.array(arr2)
arr = np.vstack((arr, arr2))
labels2=np.array(labels2)
labels = np.hstack((labels, labels2))
print(labels.shape)       
print(arr.shape)
#print(labels)    
#AfterChanging_nonfaces()

(750,)
(750, 10304)


In [127]:
#####def AfterChanging_nonfaces():

X_train = []
X_test =  []  
labels_train = []
labels_test =  []  

for i in range(0,(400+nonfaces)):
    if((i%2==0)):
       X_test.append(arr[i,:])
       labels_test.append(labels[i])
    else:
       X_train.append(arr[i,:])
       labels_train.append(labels[i])
 
print(X_test[0])
print(labels_test[0])
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES
W = LDA(arr, noOfDims=2)
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES
print("After LDA")
X_train_red, X_test_red = projectData(np.array(X_train), np.array(X_test), W)
#reg = ks().fit(X_train_red, labels_train)
#train_score = reg.score(X_train_red, labels_train)
#test_score = reg.score(X_test_red, labels_test)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, labels_train)
pred = knn.predict(X_test)
print((400+nonfaces),accuracy_score(pred,labels_test))
acc.append(accuracy_score(pred,labels_test))
#print(X_test_red)
print("Training Set Accuracy: {}\nTesting  Set Accuracy: {}".format(train_score, test_score))
print(nonfaces)
print(acc )

[157 156 153 ... 141 141 133]
1
After LDA
844 0.9976303317535545
Training Set Accuracy: 0.76
Testing  Set Accuracy: 0.7733333333333333
444
[0.9973333333333333, 0.9976303317535545]


In [ ]:
labels2 = []
arr2 =[]

for root,di,files in os.walk("/content/drive/MyDrive/Colab Notebooks/dataset3/21"): 
  for name in files:
    if name.endswith("jpg") or name.endswith("jpeg") or name.endswith("pgm") :
      root_folder = os.path.split(root) ##   
      if name.endswith("jpg") or name.endswith("jpeg"): 
        nonfaces = nonfaces + 1
        labels2.append(1)
        image1 = cv2.imread(os.path.join(root,name), 0)
      elif name.endswith("pgm") :
        image1 = imread(os.path.join(root,name))
        labels2.append(2)  
      image1.resize((92, 112))
      image1=image1.flatten(order='C')
      arr2.append(image1)
      i=i+1
      
num_nonfaces.append( nonfaces)           
arr2=np.array(arr2)
arr = np.vstack((arr, arr2))
labels2=np.array(labels2)
labels = np.hstack((labels, labels2))
print(labels.shape)       
print(arr.shape)
#print(labels)    


In [ ]:
#Call the cell of
#AfterChanging_nonfaces()

In [ ]:
plt.plot(num_nonfaces , acc)
plt.xlabel('Number of non faces')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
####################################################################################################################

In [98]:
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES
W = LDA(arr, noOfDims=2)
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES

[[124.58465011]
 [123.83069977]
 [122.79458239]
 ...
 [103.5282167 ]
 [102.64559819]
 [102.23024831]]


In [132]:
#### 7.iii
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
acc=[]
s=[]
f=[]
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, labels_train)
#before reduction
#pred = knn.predict(X_test)
#print('labels test',labels_test)
#print(pred)
knn.score(X_test, labels_test)


knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_red, labels_train)
print(knn.score(X_test_red, labels_test))
pred = knn.predict(X_test_red)
print(accuracy_score(pred,labels_test))
acc.append(accuracy_score(pred,labels_test))

pred = knn.predict(X_test_red)
#print(X_test_red,labels_test)
#print('labels test',labels_test)
#print(pred)
for i in range(len(pred)):
    if(labels_test[i] != pred[i]):
       f.append([X_test_red[i],labels_test[i]])
    else:
       s.append([X_test_red[i],labels_test[i]])
print(f[0])       

0.7867298578199052
0.7867298578199052
[array([ 8.95092952, 61.22145525]), 1]


In [ ]:
def KMC(X, k, epsilon=0.01, maxIter=100):
  # initializing the error to be max at first
  err = float('inf')
  # randomly initializing the centroids
  centroid = [X[np.random.randint(0, len(X)-1)] for i in range(k)]
  while maxIter > 0 and err >= epsilon:
    maxIter -= 1
    cluster = [[] for i in range(k)]
    # copying the old centroids
    oldCentroid = copy.deepcopy(centroid)
    # assigning each data point to closest cluster
    for x in X:
      minn, j = float('inf'), 0
      for i in range(len(centroid)):
        dist = np.linalg.norm(x-centroid[i]) ** 2
        if dist < minn: minn, j = dist, i
      cluster[j].append(x)
    # updating the centroids
    for i in range(len(cluster)):
      if len(cluster[i]) != 0: centroid[i] = sum(cluster[i]) / len(cluster[i])
    # if centroids didn't change, then terminate
    err = 0
    for i in range(len(centroid)):
      err += np.linalg.norm(centroid[i] - oldCentroid[i]) ** 2
  return cluster

In [ ]:
reg = ks().fit(X_train, labels_train)
train_score = reg.score(X_train, labels_train)
test_score = reg.score(X_test, labels_test)
print("Before LDA")
print("Training Set Accuracy: {}\nTesting  Set Accuracy: {}\n\n\n".format(train_score, test_score))

print("After LDA")
X_train_red, X_test_red = projectData(np.array(X_train), np.array(X_test), W)
reg = ks().fit(X_train_red, labels_train)
train_score = reg.score(X_train_red, labels_train)
test_score = reg.score(X_test_red, labels_test)
print('200',accuracy_score(pred,labels_test))

#print(X_test_red)
print("Training Set Accuracy: {}\nTesting  Set Accuracy: {}".format(train_score, test_score))